In [2]:
import pandas as pd, re
import io
import csv


In [ ]:
path = "/Users/clee/Documents/Lab/mlife/data/raw/diet_log_nutritionix_as_of_10-11-2025_11_10_AM.csv"

def stitch_csv_lines(p, encoding="utf-8"):
    with open(p, "r", encoding=encoding, errors="replace", newline="") as f:
        raw = f.read().splitlines()
    out, buf = [], []
    def balanced(s): return s.count('"') % 2 == 0
    for line in raw:
        if not buf:
            buf = [line]
            if balanced(line):
                out.append(line); buf = []
        else:
            buf.append(line)
            if balanced("\n".join(buf)):
                out.append("\n".join(buf)); buf = []
    if buf: out.append("\n".join(buf))
    return out

lines = stitch_csv_lines(path)

reader = csv.reader(io.StringIO("\n".join(lines)),
                    delimiter=",", quotechar='"', doublequote=True, escapechar="\\")
rows = list(reader)
header, data_rows = rows[0], rows[1:]

drop_idx = {29, 30}  # Alt Measures, Full Nutrients
keep_idx = [i for i in range(len(header)) if i not in drop_idx]
header_keep = [header[i] for i in keep_idx]
data_keep = [[r[i] if i < len(r) else "" for i in keep_idx] for r in data_rows]

df = pd.DataFrame(data_keep, columns=header_keep)

new_cols = [
    "diet_log_id","participant_name","participant_id","group","date","meal","food_name","thumb",
    "brand_name","qty","serving_type","nf_calories","nf_total_fat_g","nf_saturated_fat_g",
    "nf_cholesterol_mg","nf_sodium_mg","nf_total_carbohydrate_g","nf_dietary_fiber_g",
    "nf_sugars_g","nf_protein_g","nf_potassium_mg","nf_added_sugars_g","nf_trans_fatty_acid_g",
    "added_on","og_serving_weight_g","og_kcal","og_unit","sel_serving_qty","sel_serving_weight_g",
    "nix_brand_id","nix_brand_name","nix_item_id","nix_item_name"
]
assert len(new_cols) == df.shape[1]
df.columns = new_cols

df.head(3)
#df.to_csv('/Users/clee/Documents/Lab/mlife/data/cleaned/diet_log_fixed.csv', index=False)

,diet_log_id,participant_name,participant_id,group,date,meal,food_name,thumb,brand_name,qty,...,added_on,og_serving_weight_g,og_kcal,og_unit,sel_serving_qty,sel_serving_weight_g,nix_brand_id,nix_brand_name,nix_item_id,nix_item_name
0,152340,user user 10,10,mLife+Points,06/30/2023,Lunch,deli turkey,https://nix-tag-images.s3.amazonaws.com/855_th...,,3,...,06/30/2023 10:20:46 pm,56.7,63.5,oz,2,56.7,,,,
1,152341,user user 10,10,mLife+Points,06/30/2023,Lunch,fruit salad,https://nix-tag-images.s3.amazonaws.com/1773_t...,,2,...,06/30/2023 10:20:46 pm,162.4,96.62,cup,1,162.4,,,,
2,152342,user user 10,10,mLife+Points,06/30/2023,Dinner,hummus,https://nix-tag-images.s3.amazonaws.com/636_th...,,0.25,...,06/30/2023 10:26:14 pm,60,99.6,cup,0.25,60,,,,


In [3]:
df = pd.read_csv("/Users/clee/Documents/Lab/mlife/data/cleaned/diet_log_fixed.csv")

df['date_str'] = df['date'].astype(str)

def extract_weird_years(s):
    return re.findall(r'\b\d{3,4}\b', s)

all_years = df['date_str'].apply(extract_weird_years).explode()

year_counts = all_years.value_counts().sort_index()

print("Unique year-like tokens (including malformed ones):")
print(year_counts)

/var/folders/1f/_308mdp92hd4dnhtd2p2jgk40000gn/T/ipykernel_92234/852557771.py:1: DtypeWarning: Columns (9,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/clee/Documents/Lab/mlife/data/cleaned/diet_log_fixed.csv")


Unique year-like tokens (including malformed ones):
date_str
0202       14
1969       45
2022    97753
2023    50431
Name: count, dtype: int64


In [4]:
def extract_year_like(s):
    matches = re.findall(r'\b\d{3,4}\b', s)
    return matches[0] if matches else None

df['year_like'] = df['date_str'].apply(extract_year_like)

df['year_like'] = pd.to_numeric(df['year_like'], errors='coerce')

valid_years = [2022, 2023, 2024]

df_bad = df[~df['year_like'].isin(valid_years)]

df_bad.to_csv("/Users/clee/Documents/Lab/mlife/data/weird/01_weird_date_rows.csv", index=False)

df_filtered = df[df['year_like'].isin([2022, 2023, 2024])]

In [5]:
missing_counts = df_filtered.isnull().sum()

missing_counts = missing_counts[missing_counts > 0]

missing_counts = missing_counts.sort_values(ascending=False)

print("Missing values per column:\n")
print(missing_counts)

missing_percent = (df_filtered.isnull().mean() * 100).sort_values(ascending=False)
print("Percent missing per column:\n")
print(missing_percent[missing_percent > 0])

Missing values per column:

brand_name               110781
nix_item_name            107807
nix_item_id              107806
nix_brand_name           107806
nix_brand_id             107806
og_unit                   21741
sel_serving_weight_g       1207
og_serving_weight_g        1207
qty                          31
meal                          9
food_name                     1
og_kcal                       1
added_on                      1
sel_serving_qty               1
nf_trans_fatty_acid_g         1
nf_added_sugars_g             1
nf_sodium_mg                  1
dtype: int64
Percent missing per column:

brand_name               74.759083
nix_item_name            72.752119
nix_brand_id             72.751444
nix_item_id              72.751444
nix_brand_name           72.751444
og_unit                  14.671624
og_serving_weight_g       0.814528
sel_serving_weight_g      0.814528
qty                       0.020920
meal                      0.006074
og_kcal                   0.000675
n

In [34]:
num_unique_ids = df_filtered['participant_id'].nunique()
print(f"Number of unique participant_id values: {num_unique_ids}")

unique_ids = df_filtered['participant_id'].unique()
print(unique_ids)

Number of unique participant_id values: 106
[ 10 100 101 102 103 104 105 106 107 108 109  11 110 111 112 113 114 115
 116 117 118 119  12 120 121 122  13  15  16  17  18  19  20  21  22  23
  24  25  26  27  29  30  31  32  33  35  36  37  38  40  41  42  43  44
  45  46  47  48  49  50  51  52  53  54  55  56  57  58  60  61  62  63
  65  66  67  68  69   7  70  71  72  73  74  76  77  78  79   8  80  81
  82  83  84  86  87  88  89   9  90  91  92  94  95  96  97  98]


In [8]:
group_counts = df_filtered['group'].value_counts(dropna=False)
print("Counts for 'group':\n", group_counts, "\n")

meal_counts = df_filtered['meal'].value_counts(dropna=False)
print("Counts for 'meal':\n", meal_counts)

Counts for 'group':
 group
mLife+Points       91277
mLife (control)    56907
Name: count, dtype: int64 

Counts for 'meal':
 meal
Dinner       43731
Breakfast    43263
Lunch        39557
Snack        21624
NaN              9
Name: count, dtype: int64
